<center>
   Go back to the <a href="https://github.com/kadrlica/skymap/tree/master/examples/README.md">Index</a>
</center>

# Tutorial 4: Skymap and DECam

In this tutorial we show how `cartosky` can interface with different astronomical surveys and instruments. Here, we specifically examine some of the features for the Dark Energy Survey (DES; [DES Collaboration 2005](https://arxiv.org/abs/astro-ph/0510346)) and the Dark Energy Camera (DECam; [Flaugher et al. 2015](https://arxiv.org/abs/1504.02900)).


In [ ]:
# Basic notebook imports
%matplotlib inline

import os
import matplotlib
import pylab as plt
import numpy as np
import healpy as hp

We start by importing several classes from the skymap module and setting a few constants that we will use in this example. By this point, these modules and classes should be familiar.

In [ ]:
import cartosky
from cartosky.constants import FIGSIZE,SCALE,DPI,DECAM
from cartosky.utils import get_datadir
from cartosky import SurveyMcBryde
from cartosky import DESSkymap

NSIDE = 64

When plotting DES data, it is often convenient to use an inset colorbar placed between the SPT and Stripe82 regions. This is demonstrated with `DESSkymap` below.

In [ ]:
filename = os.path.join(get_datadir(),'des-round13-poly.txt')
vertices = np.loadtxt(filename)
vertices[:,0] -= 360*(vertices[:,0] > 180)
path = matplotlib.path.Path(vertices)

hpxmap = hp.UNSEEN*np.ones(hp.nside2npix(NSIDE))
pixels = np.arange(hp.nside2npix(NSIDE))
ra,dec = hp.pix2ang(NSIDE,pixels,lonlat=True)
ra -= 360*(ra > 180)
sel = path.contains_points(np.array([ra,dec]).T)

hpxmap[sel] = pixels[sel]

fig = plt.figure(figsize=FIGSIZE)
smap = DESSkymap()
smap.draw_hpxmap(hpxmap,xsize=400)
smap.draw_inset_colorbar(ticks=[2.6e4,4.2e4])
smap.draw_des13(edgecolor='k',alpha=0.5)

We have implemented several tools to plot the DECam focal plane. The first uses a quick and dirty scatter plot. Beware, the scatter points do not scale if the figure or axis is resized!

In [ ]:
ra = dec = np.arange(-30,30)
band = 20*['g']+20*['r']+20*['z']
fields = np.rec.fromarrays([ra,dec,band],dtype=[('ra',float),('dec',float),('band',object)])

# Careful, the scatter point size depends on the figure and is currently not accurate
plt.figure(figsize=FIGSIZE,dpi=DPI)
smap = SurveyMcBryde()
smap.draw_fields(fields,alpha=0.5,s=30)

# The way to check that the size is ok is to use a tissot with prescribed radius
for x,y in [(-30,-30),(0,0),(30,30)]:
    smap.tissot(x,y,DECAM,edgecolor='k',facecolor='none')

The second plotting uses a much more accurate (but slower) `PolygonCollection` to represent the focal plane at CCD level.

In [ ]:
ra,dec = 45,-45
radius = 1.5
delta = 1.0

plt.figure(figsize=(8,8))
                               
m = cartosky.Skymap(projection='cyl', celestial=False, gridlines=False)
llcrnrlon,urcrnrlon = ra+2*radius, ra-2*radius
llcrnrlat,urcrnrlat = dec-2*radius, dec+2*radius
m.ax.set_extent([llcrnrlon,urcrnrlon,llcrnrlat,urcrnrlat])

# Can plot individual fields                                                                      
m.draw_focal_planes([ra+delta/2],[dec-delta/2],color='g')
# Or as arrays                                                                                    
m.draw_focal_planes([ra,ra-delta,ra-delta],[dec,dec+delta,dec-delta],color='r')

# Draw the approximate focal plane
m.tissot(ra,dec,1.1,ec='g',fc='none')

## Draw the grid lines                                                                            
draw_labels = True
xlocs = np.linspace(llcrnrlon,urcrnrlon,5)
ylocs = np.linspace(llcrnrlat,urcrnrlat,5)
m.ax.gridlines(draw_labels=draw_labels,
               xlocs=xlocs,ylocs=ylocs,
               linestyle=':')

# ADW: This doesn't work (removes y-labels)...                                                    
#m.ax.invert_xaxis()                                                                              

plt.title('DECam Focal Planes')

<center>
   Go back to the <a href="https://github.com/kadrlica/skymap/tree/master/examples/README.md">Index</a>
</center>